# Pet-project Oyster

## Описание проекта

TODO

https://habr.com/ru/sandbox/197522/
https://proverkacheka.com/cabinet/checks

## Описание данных

TODO  
qrraws.csv  
Bills_(year).csv  
Goods_(year).csv  

## Импорт библиотек

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import requests
import json
import uuid
import datetime
import os


## Константы

In [2]:
URL = "https://proverkacheka.com/api/v1/check/get"
TOKEN = "25602.ZzjFe81k1NGymEnRU"
QRRAWS_PATH = "./sources/qrraws.csv"
SOURCES_DIR = "./sources"
# BILLS_SOURCE_NAME = "Bill"
# GOODS_SOURCE_NAME = ""

## Вспомогательные методы

### Для подгрузки данных

In [3]:
# ОТЛАЖЕНО
def load_qrraws_data():
    """
    Метод, который выгружает qrraws_data и возвращает df 
    и выводит справочнуюинформацию о нём
    """
    
    qrraws_data = pd.read_csv(QRRAWS_PATH, sep=';')
    qrraws_data = qrraws_data.set_index('id')
    
    # Вывод справочной информации
#     print(qrraws_data.head(3))
#     qrraws_data.info()
    
    return qrraws_data

In [4]:
# ОТЛАЖЕНО
def get_bill_data(qrraw, state):
    """
    Метод отправляет обращения на сервис проверки чеков, если по этому чеку ещё нет записи в Bills_year.csv (state != 1),
    после этого, если всё хорошо (код ответа == 1), обращается к методу записи результата в Bills_year.csv. 
    При любом исходе метод возвращает новый state для текущего qrraw.
    
    Пример qrraw:
    "t=20200924T1837&s=349.93&fn=9282440300682838&i=46534&fp=1273019065&n=1"
    """
    
    if state == 1:
        return state
    
    url = URL
    data = {
        "token": TOKEN,
        "qrraw": qrraw,
    }
    r = requests.post(url, data=data)
    result = json.loads(r.text)
    
    if result['code'] != 1:
        print(f"ERR {result['code']}, {result['data']} qrraw = {qrraw}")
    else:
        write_bill_data(result, r.text)
    return result['code']

# TODO проработать расшифровку кодов ответов. Обрабатываем по-новой те записи, где код не 1
# ERR 3, Превышено количество обращений по чеку.
# ERR 3, Превышено количество обращений для учетной записи пользователя.
# ERR 4, Не вышло время ожидания перед повторным запросом.
# ERR 5, Нет информации по чеку (прочее).

In [5]:
# ОТЛАЖЕНО
def write_bill_data(result, text_version):
    """
    Метод добавляет запись в таблицу Bills_year.csv. Происходит автоматическое создание файла по году, если такого ещё нет.
    При первичном создании в bill_data.to_csv header=True, при последующих False, чтобы не дублировались заголовки.
    Метод возвращает 200, если в процессе работы не произошло ошибок.
    """
    
    records_year = pd.to_datetime(result['data']['json']['dateTime'], format='%Y-%m-%dT%H:%M:%S').year
    record_uuid = uuid.uuid4()
    qrraw = result['request']['qrraw']
    scan_date = pd.to_datetime(datetime.date.today().isoformat(), format='%Y-%m-%d')
    day_of_purchase = pd.to_datetime(result['data']['json']['dateTime'], format='%Y-%m-%dT%H:%M:%S')
    
    bill_data = pd.DataFrame(
        [[record_uuid, qrraw, text_version, scan_date, day_of_purchase, 0]], 
        columns=['id', 'qrraw', 'full_data', 'scan_date', 'day_of_purchase', 'parsed']
    )
    file_path = f'./sources/Bills_{records_year}.csv'
    header_flag = check_file(file_path) == False
    
    bill_data.to_csv(file_path, mode='a', header=header_flag, index=False)
    return 200

In [6]:
# ОТЛАЖЕНО
def check_file(file_path):
    """
    Метод проверяет, существует ли файл по указанному пути. Возвращает, соответственно True или False.
    """
    
    if os.path.exists(file_path):
        return True
    else:
        return False

In [21]:
def load_goods_data(bill_id, full_data, parsed):
    """
    Метод, который принимает инфомацию полную по чеку и делит его на товары из этого чека.
    После этого добавляет записи в источник Goods_year.csv и возврашает 1 в качетве отметки о выполнении.
    """
    if parsed == 1:
        return parsed
    
    full_data = json.loads(full_data)
    records_year = pd.to_datetime(full_data['data']['json']['dateTime'], format='%Y-%m-%dT%H:%M:%S').year
    shop_name = full_data['data']['json']['retailPlace']
    shop_address = full_data['data']['json']['metadata']['address']
    day_of_purchase = pd.to_datetime(full_data['data']['json']['dateTime'], format='%Y-%m-%dT%H:%M:%S')
    load_date = pd.to_datetime(datetime.date.today().isoformat(), format='%Y-%m-%d')
    
    for item in full_data['data']['json']['items']:
        name = item['name']
        price = item['price']
        quantity = item['quantity']
        full_cost = item['sum']
        category = detect_category(name) # TODO в разработке отдельный метод-классификатор, пока возвращает пустую строку всегда
        
        good_data = pd.DataFrame(
            [[bill_id, name, category, price, quantity, full_cost, shop_name, shop_address, day_of_purchase, load_date]], 
            columns=['bill_id', 'name', 'category', 'price', 'quantity', 'full_cost', 'shop_name', 'shop_address', 'day_of_purchase', 'load_date']
        )
        file_path = f'./sources/Goods_{records_year}.csv'
        header_flag = check_file(file_path) == False
        
        good_data.to_csv(file_path, mode='a', header=header_flag, index=False)
    return 1

In [22]:
def start_goods_loader():
    """
    Метод, который проходится по Bills_year.csv и там, где parsed != 1, парсит чек по товарам метоом load_goods_data().
    После парса чека, обновляет параметр parsed=1 и обновляет источник Bills_year.csv.
    """
    # Получаем список файлов
    files = os.listdir(SOURCES_DIR)
    found = [file for file in files if 'Bills' in file]
    years_list = list(map(lambda x: x.split('.')[0].split('_')[1], found))
    
    for year in years_list:
        bills_data = pd.read_csv(f'./sources/Bills_{year}.csv')
    
        # Запуск загрузки в Goods_year.csv
        bills_data['parsed'] = bills_data.apply(lambda x: load_goods_data(x.id, x.full_data, x.parsed), axis=1)
        
        # Перезапись данных в Bills_year.csv, туда отправляются обновлённые коды стейта загрузки в Goods_year.csv
        bills_data.to_csv(f'./sources/Bills_{year}.csv', header=True, index=False)
        display(f'Обработка данных из Bills_{year}.csv COMPLETED')

In [9]:
def detect_category(good_name):
    """
    TODO
    """
    #TODO
    return ''

In [24]:
def new_data_processor(new_qrraws):
    '''
    Этот метод состоит из 3-х этапов:
    1. STAGE: new_qrraws -> qrraws.csv
    2. STAGE: qrraws.csv -> Bills_year.csv
    3. STAGE: Bills_year.csv -> Goods_year.csv

    Вызов каждого из них отображается в выводе. Более подробное описание есть у кажого этапа.
    '''


    # $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ qrraws.csv $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    '''
    Добавляем новые qrraw в qrraws.csv, дублей не будет, если попробовать ввести уже имеющиеся
    '''
    print(f"\nSTAGE: new_qrraws -> qrraws.csv ****************************************************************************************")
    # Вытаскиваем уже имеющиеся в источнике данные
    qrraws_data = load_qrraws_data()
    
    # Формирование валидного массива словарей из new_qrraws 
    arr_of_additions = []
    for new_qrraw in new_qrraws:
        if qrraws_data.query("qrraws == @new_qrraw")['qrraws'].count() > 0:
            print(f'Уже есть в qrraws: {new_qrraw}')
        else:
            arr_of_additions.append({'qrraws':new_qrraw, 'state':0})
    
    # Преобразование массива словарей new_qrraws в df и присоединение его к основному qrraws_data
    addition = pd.DataFrame(arr_of_additions)
    qrraws_data=qrraws_data.append(addition,ignore_index=True)
    
    # Профилактическая очистка от дубликатов qrraw в qrraws_data
    qrraws_data = qrraws_data.drop_duplicates(['qrraws'])
    # Запись новых данных в источник qrraws
    qrraws_data.to_csv(QRRAWS_PATH, sep=';', index=True, index_label='id')
    
    # Проверка, остались ли дубликаты
    qrraws_data[qrraws_data.duplicated(['qrraws'])]
    # $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ qrraws.csv $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    # $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ Bills_year.csv $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    '''
    Запуск выгрузки данных о чеках в Bills_year.csv по qrraws. 
    Запросы идут только на записи, где state !=1, иначе считаем, что по этому чеку уже есть данные в Bills_year.csv. 
    Тут же будет вывод о qrraws, для которых не получилось добавить данные по чекам из-за ошибок
    
    При повторных запусках кода выше не будет никаких лишних дублей
    '''
    print(f"\nSTAGE: qrraws.csv -> Bills_year.csv ****************************************************************************************")
    
    # Считываем данные из хранилища qrraws
    qrraws_data = load_qrraws_data()
    
    # ****************************************************************************************
    # На случай, если нужно полностью восстановить чеки из qrraws, сбрасываем стейты
    # qrraws_data['state'] = 0
    # qrraws_data.to_csv(QRRAWS_PATH, sep=';', index=True, index_label='id')
    # ****************************************************************************************
    
    # Запуск загрузки в Bills_year.csv
    qrraws_data['state'] = qrraws_data.apply(lambda x: get_bill_data(x.qrraws, x.state), axis=1)
    # Перезапись данных в qrraws.csv, туда отправляются обновлённые коды стейта загрузки в Bills_year.csv
    qrraws_data.to_csv(QRRAWS_PATH, sep=';', index=True, index_label='id')
    
    # ****************************************************************************************
    # На случай, если нужно убрать записи, где код не 0 или не 1
    # qrraws_data = qrraws_data.query("state == 0 or state == 1")
    # qrraws_data = qrraws_data.reset_index(drop= True)
    # qrraws_data
    # qrraws_data.to_csv(QRRAWS_PATH, sep=';', index=True, index_label='id')
    # ****************************************************************************************
    # $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ Bills_year.csv $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    # $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ Goods_year.csv $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    '''
    Запуск выгрузки данных о чеках в Goods_year.csv по Bills_year.csv. 
    Запросы идут только на записи, где state !=1, иначе считаем, что по этому чеку уже есть данные в Goods_year.csv.
    '''
    print(f"\nSTAGE: Bills_year.csv -> Goods_year.csv ****************************************************************************************")
    
    start_goods_loader()
    # $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ Goods_year.csv $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

### Для аналитики

## Раздел подгрузки данных. Ручное добавление в qrraws

Пока не реализовано ничего для автоматизации пополнения qrraws, добавляем записи в полуручном режиме:  
- вручную или через сканер получить qrraw-ы с чеков ([Для сканирования qr с компьютера](#https://code-qr.ru/online))
- записать эти qrraw-ы в массив new_qrraws ниже, как строки

In [25]:
# Тут формируем массив с новыми qrraw. Если внести qrraw, который уже есть в таблице, дубля не будет, т.к. метод его пропустит
new_qrraws = [
't=20240125T1750&s=134.99&fn=7282440500188699&i=71493&fp=2442920498&n=1',
't=20240125T1737&s=415.83&fn=7282440500188723&i=61218&fp=10234313&n=1',
]

new_data_processor(new_qrraws)


STAGE: new_qrraws -> qrraws.csv ****************************************************************************************
Уже есть в qrraws: t=20240125T1750&s=134.99&fn=7282440500188699&i=71493&fp=2442920498&n=1
Уже есть в qrraws: t=20240125T1737&s=415.83&fn=7282440500188723&i=61218&fp=10234313&n=1

STAGE: qrraws.csv -> Bills_year.csv ****************************************************************************************
ERR 3, Превышено количество обращений по чеку. qrraw = t=20231029T1838&s=909.09&fn=7281440501305730&i=3739&fp=2449765930&n=1

STAGE: Bills_year.csv -> Goods_year.csv ****************************************************************************************


'Обработка данных из Bills_2024.csv COMPLETED'

'Обработка данных из Bills_2023.csv COMPLETED'

# TODO, тут дальше песочница для метода разметки категорий. **Сейчас категории проставляются руками, чтобы дальше на этом обучить модель**

In [12]:
# '''
# Тут создание общего пула товаров по всем годам.
# Инициализируем full_products_df
# '''
# # Получаем список файлов
# files = os.listdir(SOURCES_DIR)
# found = [file for file in files if 'Goods' in file]
# years_list = list(map(lambda x: x.split('.')[0].split('_')[1], found))

# full_products_df = pd.DataFrame()

# for year in years_list:
#     goods_data = pd.read_csv(f'./sources/Goods_{year}.csv')
#     display(f'Обработка данных из Goods_{year}.csv ...')
# #     display(goods_data.head(1))
# #     display(goods_data.info())
#     full_products_df = full_products_df.append(goods_data,ignore_index=True)
#     display(f'Обработка данных из Goods_{year}.csv COMPLETED')

In [13]:
# full_products_df.sample(10)

In [14]:
# from sklearn.svm import SVC
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OneHotEncoder

# from sklearn.model_selection import train_test_split

# data = pd.read_csv('./sources/Goods_2023.csv')
# display(data.sample(3))

# X = data.drop(columns=['category', 'bill_id', 'full_cost', 'shop_address', 'day_of_purchase', 'load_date', 'category_id'])
# y = data['category']

# display(X.head(5))
# display(y.head(5))

In [15]:
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import LabelEncoder

# RANDOM_STATE = 42

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, 
#     test_size=0.25, 
#     stratify=y,
#     random_state=RANDOM_STATE)
# А дальше задача для работы с текстами
# https://radimrehurek.com/gensim/models/doc2vec.html
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#sphx-glr-auto-examples-tutorials-run-doc2vec-lee-py

In [16]:
# pd.options.display.max_rows = 1000
# full_products_df['name'].value_counts()

In [17]:
# url = URL
# data = {
#     "token": TOKEN,
#     "qrraw": 't=20230721t1636&s=715.21&fn=9961440300305579&i=68881&fp=220365927&n=1',
# }
# r = requests.post(url, data=data)
# result = json.loads(r.text)
# result['data']['json']

## Раздел анализа данных TODO

Текущие возможные категории:
- Соусы
- Чай/кофе
- Хлеб
- Хлебобулочные продукты
- Кондитерские изделия
- Сладости и гадости
- Пищевые жиры
- Яйца
- Мясо и мясопродукты
- Рыба и рыбопродукты
- Молоко и молочные продукты
- Крупы
- Макаронные изделия
- Зелень
- Овощи
- Фрукты и ягоды
- Алкоголь
- Орехи
- Грибы
- Консервы
- Гигиена
- Хозяйственное
- Алкоголь
- Другое
- Полуфабрикаты
- Пряности

In [18]:
data_2023 = pd.read_csv('./sources/Goods_2023.csv')
data_2024 = pd.read_csv('./sources/Goods_2024.csv')

In [19]:
data_2023['category'].value_counts()

Овощи                         182
Молоко и молочные продукты    130
Фрукты и ягоды                118
Мясо и мясопродукты            85
Сладости и гадости             70
Хлебобулочные продукты         48
Хлеб                           48
Рыба и рыбопродукты            41
Хозяйственное                  22
Макаронные изделия             21
Пищевые жиры                   20
Крупы                          17
Яйца                           16
Консервы                       16
Пряности                       14
Соусы                          14
Другое                         13
Чай/кофе                       11
Орехи                          10
Грибы                          10
Зелень                          9
Алкоголь                        8
Полуфабрикаты                   8
Гигиена                         4
Name: category, dtype: int64

In [20]:
data_2024['category'].value_counts()

Series([], Name: category, dtype: int64)